# 데이터베이스에서 가져오기

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/rickiepark/hg-da/blob/main/02-1.ipynb"><img src="https://jupyter.org/assets/share.png" width="61" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-da/blob/main/02-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

## SQLite

In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('ns_lib.db')

In [3]:
import gdown

gdown.download('https://bit.ly/3RhoNho', 'ns_202104.csv', quiet=False)

Downloading...
From: https://bit.ly/3RhoNho
To: /content/ns_202104.csv
100%|██████████| 57.6M/57.6M [00:01<00:00, 39.9MB/s]


'ns_202104.csv'

**이전에 만든 nslib_book 테이블이 있다면 먼저먼저 삭제해 주세요.**

In [4]:
c = conn.cursor()

c.execute("CREATE TABLE nslib_book \
          (name TEXT, author TEXT, borrow_count INTEGER)")

In [5]:
c.execute("CREATE TABLE IF NOT EXISTS nslib_book \
          (name TEXT, author TEXT, borrow_count INTEGER)")

In [6]:
c.execute("DROP TABLE nslib_book")

In [7]:
c.execute("CREATE TABLE nslib_book \
          (name TEXT, author TEXT, borrow_count INTEGER)")

## 테이블에 남산 도서관 데이터 추가하기

In [8]:
import pandas as pd

ns_df = pd.read_csv('ns_202104.csv', low_memory=False)
ns_df.head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자,Unnamed: 13
0,1,인공지능과 흙,김동훈 지음,민음사,2021,9788937444319,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
1,2,가짜 행복 권하는 사회,김태형 지음,갈매나무,2021,9791190123969,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
2,3,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,블랙피쉬,2021,9788968332982,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
3,4,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",문학세계사,2021,9788970759906,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
4,5,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,김영사,2021,9788934990833,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN


In [9]:
for index, row in ns_df.iterrows():
    c.execute("INSERT INTO nslib_book (name,author,borrow_count) \
              VALUES (?,?,?)", (row['도서명'], row['저자'], row['대출건수']))

In [10]:
for index, row in ns_df.iterrows():
    pass

In [11]:
book_df = ns_df[['도서명','저자','대출건수']]
book_df.head()

,도서명,저자,대출건수
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


In [12]:
book_df.columns = ['name', 'author', 'borrow_count']
book_df.head()

,name,author,borrow_count
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


In [13]:
book_df.to_sql('nslib_book', conn, if_exists='replace', index=False)

## 테이블에 남산 도서관 데이터 읽기

In [14]:
c.execute("SELECT * FROM nslib_book")

In [15]:
c.fetchone()

('인공지능과 흙', '김동훈 지음', 0)

In [16]:
c.fetchone()

('가짜 행복 권하는 사회', '김태형 지음', 0)

In [17]:
c.fetchmany(3)

[('나도 한 문장 잘 쓰면 바랄 게 없겠네', '김선영 지음', 0),
 ('예루살렘 해변', '이도 게펜 지음, 임재희 옮김', 0),
 ('김성곤의 중국한시기행 : 장강·황하 편', '김성곤 지음', 0)]

In [18]:
all_rows = c.fetchall()

In [19]:
book_df = pd.DataFrame(all_rows)
book_df.head()

,0,1,2
0,처음 읽는 음식의 세계사,"미야자키 마사카츠 지음, 한세희 옮김",0
1,아르센 벵거 자서전 My Life in Red and White,"아르센 벵거 지음, 이성모 옮김",0
2,쓰고 달콤한 직업,천운영 지음,0
3,일잘러의 노션 100% 활용법,제레미 강 지음,0
4,타이난 골목 노포 산책,"천구이팡 지음, 심혜경 외 옮김",0


In [20]:
book_df = pd.read_sql_query("SELECT * FROM nslib_book", conn)
book_df.head()

,name,author,borrow_count
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


In [21]:
book_df = pd.read_sql_table('nslib_book', 'sqlite:///ns_lib.db')
book_df.head()

,name,author,borrow_count
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


## 통계 함수와 정렬 기능으로 데이터 탐색하기

In [22]:
len(book_df)

401682

In [23]:
c.execute("SELECT count(*) FROM nslib_book")
c.fetchone()

(401682,)

In [24]:
c.execute("SELECT sum(borrow_count) FROM nslib_book")
c.fetchone()

(4400145,)

In [25]:
c.execute("SELECT avg(borrow_count) FROM nslib_book")
c.fetchone()

(10.95429966988812,)

In [26]:
c.execute("SELECT * FROM nslib_book ORDER BY borrow_count DESC LIMIT 10")
c.fetchall()

[('사피엔스 :유인원에서 사이보그까지, 인간 역사의 대담하고 위대한 질문 ', '유발 하라리 지음 ;조현욱 옮김', 1468),
 ('해커스 토익:Listening', 'David Cho 지음', 1065),
 ('7년의 밤 :정유정 장편소설 ', '정유정 저', 683),
 ('냉정과 열정사이:Blu', '츠지 히토나리 지음;양억관 옮김', 524),
 ('남한산성:김훈 장편소설', '김훈 지음', 501),
 ('해리포터와 혼혈왕자', '조앤 K. 롤링 지음;최인자 옮김', 451),
 ('해커스 토익:Listening', 'David Cho 지음', 440),
 ('다빈치 코드', '댄 브라운 지음;양선아 옮김', 440),
 ('신:베르나르 베르베르 장편소설', '베르나르 베르베르 지음;이세욱 옮김', 432),
 ('경제학 콘서트', '팀 하포드 지음;김명철 옮김', 425)]

In [27]:
c.close()
conn.close()